# Topic and word list from a working paper by Sophie Stone of the Dartmouth College Economics Department



## https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
# Dependencies
# !pip install "calcbench-api-client[Pandas, Backoff, BeautifulSoup]" sentence-transformers tqdm qgrid ipywidgets

In [ ]:
import warnings
from collections import Counter

import pandas as pd
import itertools
from tqdm.notebook import tqdm
import qgrid

import calcbench as cb


cb.enable_backoff()
tqdm.pandas(desc="progress")

In [ ]:
categories = {
    "environmental": """biodiversity, carbon, carbon negative, carbon neutral, carbon zero, clean-up, clean, clean tech, climate change, climate positive, contamination, discharge, emission, energy-efficient, environmental, environmental risk, fuel, fuel efficiency, green, greenhouse gas, hazardous, low carbon, natural resource, net-zero emission, pollution, remediation, sustainability, sustainable, toxic, waste, water, zero carbon, zero net carbon""",
    "social": """accident, antiracism, consumer protection, customer privacy, employee relation, equal, equal pay, equity, gender equality, health, human right, justice, labor relation, labor standard, racial equity, racial awareness, racial justice, working condition""",
    "governance": """advocacy, antitrust, board independence, code of ethic, compensation, corporate culture, corporate governance, corruption, governance risk, justice, political lobbying, scandal, shareholder right, stability, stewardship, transparency""",
}

category_embeddings = {
    category: set(w.strip().lower() for w in category_words.split(',')) for (category, category_words) in categories.items()
}

In [ ]:
with tqdm() as progress_bar:
    disclosures = cb.document_dataframe(
        company_identifiers=cb.tickers(index="DJIA"),
        disclosure_names=["ManagementsDiscussionAndAnalysis", "RiskFactors"],
        all_history=True,
        period_type="annual",
        progress_bar=progress_bar,
        # entire_universe=True,
    )

In [ ]:
def get_contents(d):
    try:
        return d.get_contents_text()
    except Exception as e:
        print(f"Exception getting {d} \n{e}")


disclosure_contents = disclosures.progress_applymap(
    get_contents, na_action="ignore"
).fillna("")

In [ ]:
disclosure_contents = pd.read_pickle("disclosure_contents.pkl")

In [ ]:
def distance(word_counts, category_words):
    word_counts =[(word, count) for word, count in word_counts.items() if word in category_words]
    return sum(count for _, count in word_counts)

In [ ]:
distances = []
for _, category_words in tqdm(category_embeddings.items()):
    category_distances = disclosure_embeddings.progress_applymap(lambda word_counts:distance(word_counts, category_words)    )
    distances.append(category_distances)
esg_distances = pd.concat(
    distances, axis=1, keys=[category for category, _ in categories.items()]
)

In [ ]:
esg_distances